# Fitting Description

## Mount Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content/drive/MyDrive/digital_breakthrough/task_3

/content/drive/MyDrive/digital_breakthrough/task_3


In [ ]:
!pip install -r requirements.txt

In [1]:
cd /content/drive/MyDrive/digital_breakthrough/task_3

/content/drive/MyDrive/digital_breakthrough/task_3


In [24]:
from transformers import TFRobertaModel, RobertaModel, TFGPT2LMHeadModel, TFGPT2Model, RobertaTokenizer, BertTokenizer

In [18]:
gpt_encoder = TFGPT2Model.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2", from_pt=True, output_hidden_states=True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2Model: ['transformer.h.11.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'lm_head.weight', 'transformer.h.0.attn.masked_bias', 'transformer.h.10.attn.masked_bias']
- This IS expected if you are initializing TFGPT2Model from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2Model from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All t

In [8]:
transformer_encoder = TFRobertaModel.from_pretrained(
    "DeepPavlov/rubert-base-cased-sentence", 
    from_pt=True, 
    output_hidden_states=True
)

You are using a model of type bert to instantiate a model of type roberta. This is not supported for all configurations of models and can yield errors.


Downloading:   0%|          | 0.00/711M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFRobertaModel.

All the weights of TFRobertaModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


In [25]:
tokenizer = BertTokenizer.from_pretrained("DeepPavlov/rubert-base-cased-sentence")

Downloading:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

## Load Data

In [ ]:
import sys
import numpy as np
import pandas as pd
from tqdm import tqdm
sys.path.append('.')
from definitions import DATA_PATH, TRAIN_IMAGES, DOWNLOADED_TRAIN_IMAGES
from airotica.utils import read_image

In [ ]:
train = pd.read_csv(DATA_PATH / 'train.csv')
train_url_only = pd.read_csv(DATA_PATH / 'train_url_only.csv')
train_url_loaded_images = pd.read_csv(DATA_PATH / 'train_loaded_images.csv')
test = pd.read_csv(DATA_PATH / 'test.csv')
sample_submission = pd.read_csv(DATA_PATH / 'sample_submission.csv')

In [ ]:
train_labels = train.typology.unique()
typology_to_label = dict(zip(sorted(train_labels), range(len(train_labels))))
label_to_typology = {v:k for k, v in typology_to_label.items()}

In [ ]:
MAPPER = {0: 0, 1:1, 2:2, 3:3, 4:4, 5:5, 6:6, 
          7:7, 8:8, 9:9, 10:10, 11:12, 12:13, 13:14,
          -1: -1
          }

 ## FitImages

In [ ]:
import logging
from airotica.airotica import detectron
from src.predict import prediction

%matplotlib inline
%load_ext autoreload
%autoreload 2

log = logging.getLogger(__name__)

{"asctime": "2021-08-04 08:41:35", "name": "matplotlib.pyplot", "filename": "pyplot.py", "levelname": "DEBUG", "message": "Loaded backend module://ipykernel.pylab.backend_inline version unknown."}
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
images_prediction = detectron(config_name='config5.yml', 
                              save_path='./sub/predictv3.csv',
                              model_path='./EXPERIMENTS/full_res/model_0001_0.857586.pth',
                              model_type='resnext101_32x8d',
                              save_result=False,
                              return_result=True
                              )

  0%|          | 0/1223 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
100%|█████████▉| 1219/1223 [05:20<00:00,  4.86it/s]

In [ ]:
pred = prediction('predict.yaml')

tokenizing:   0%|          | 0/676 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2190: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
tokenizing: 100%|██████████| 676/676 [00:00<00:00, 8474.13it/s]


3/3 [==============================] - 1s 48ms/step


tokenizing: 100%|██████████| 676/676 [00:00<00:00, 7472.14it/s]


3/3 [==============================] - 1s 48ms/step


In [ ]:
images_prediction.loc[:, ['label']] = images_prediction['sign_0'].map(MAPPER)
images_prediction.loc[:, ['result']] = images_prediction['label'].map(label_to_typology)
im_test = images_prediction[images_prediction.sign_0 != -1]

In [ ]:
sample_submission.loc[pred.index, 'typology'] = pred.values
sample_submission.loc[im_test.index, 'typology'] = im_test.result

In [ ]:
sample_submission.to_csv('./sub/imbert.csv', index=False)